In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

from sklearn.utils import shuffle

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/train_reviews.csv',encoding='ISO-8859-1')

<ipython-input-6-4bc24dbd3b55>:1: DtypeWarning: Columns (5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/gdrive/MyDrive/train_reviews.csv',encoding='ISO-8859-1')


In [ ]:
df.shape # observate the shape of the trainset

(392426, 65)

In [ ]:
df=df.iloc[:,0:11] # keep valid column

In [ ]:
df.info() # Look at the data type

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392426 entries, 0 to 392425
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   review_title      392069 non-null  object
 1   review_body       392382 non-null  object
 2   review_date       392426 non-null  object
 3   review_rating     392425 non-null  object
 4   number_of_photos  392426 non-null  object
 5   helpful_vote      392426 non-null  object
 6   reviewer_ID       392426 non-null  object
 7   fake_asin         392426 non-null  object
 8   fake_review       392426 non-null  object
 9   product_ID        392414 non-null  object
 10  review_ID         392123 non-null  object
dtypes: object(11)
memory usage: 32.9+ MB


In [ ]:
# Check rows whether misaligned data or outliers exist
print(df['review_rating'].value_counts())
print(df['number_of_photos'].value_counts())
print(df['fake_review'].value_counts())

In [ ]:
df = df.astype(str)

In [ ]:
# delete rows where review rating is not between 0 and 5.
df['length'] = df.review_rating.str.len()
df = df[df.length ==1]

In [ ]:
# delete rows where review body and review date are stuck together.
df=df[df["fake_review"].isin(['0','1'])]

In [ ]:
# check again whether there are rows with misaligned data. we found that each numeric variables has reasonable rariance. 
print(df['review_rating'].value_counts())

5    261693
4     48268
1     39726
3     23626
2     18562
Name: review_rating, dtype: int64


In [ ]:
print(df['number_of_photos'].value_counts())

0     374258
1      10776
2       3965
3       1661
4        659
5        252
6        157
7         62
8         30
9         16
10        14
12         9
11         7
14         2
13         2
15         2
16         1
25         1
18         1
Name: number_of_photos, dtype: int64


In [ ]:
print(df['helpful_vote'].value_counts())

0      322643
1       38074
2       10553
3        4742
4        2725
        ...  
355         1
261         1
531         1
433         1
192         1
Name: helpful_vote, Length: 460, dtype: int64


In [ ]:
print(df['fake_asin'].value_counts())

0    336549
1     55326
Name: fake_asin, dtype: int64


In [ ]:
print(df['fake_review'].value_counts())

0    371876
1     19999
Name: fake_review, dtype: int64


In [ ]:
#delete "length" column since it is unncessary.
df=df.drop('length',axis=1)

In [ ]:
# format the data into appropriate types
df['review_rating']=df['review_rating'].astype('int')
df['number_of_photos']=df['number_of_photos'].astype('int')
df['helpful_vote']=df['helpful_vote'].astype('int')
df['fake_review']=df['fake_review'].astype('int')
df['fake_asin']=df['fake_asin'].astype('int')
df['product_ID']=df['product_ID'].astype('float').astype('int')
df['review_ID']=df['review_ID'].astype('float').astype('int')

In [ ]:
#combine review title and review body
df['review'] = df['review_title'] + ' '+df['review_body']

In [ ]:
# replace review columns null with NA
df['review'] = df['review'].fillna('NA')

In [ ]:
# convert 'date' column to datetime object
df['review_date'] = pd.to_datetime(df['review_date'])

# create new 'year' column extracted from 'date' column
df['year'] = df['review_date'].dt.year

df['month'] = df['review_date'].dt.month

df['day'] = df['review_date'].dt.day


In [ ]:
# create "is_weekday" to represent whether reviews are on weekends.
df['is_weekday'] = pd.to_datetime(df['review_date']).dt.dayofweek.apply(lambda x: 1 if x < 5 else 0)

In [ ]:
# drop the column: review_title, review_body, review_date
df = df.drop(["review_title", "review_body"], axis = 1)

In [ ]:
df.head()

,review_date,review_rating,number_of_photos,helpful_vote,reviewer_ID,fake_asin,fake_review,product_ID,review_ID,review,year,month,day,is_weekday
0,2020-04-13,5,0,0,380263,1,1,389,34510,Super product My husband has a hard time sleep...,2020,4,13,1
1,2020-08-29,5,0,0,845137,0,0,99,381688,Very sturdy Rving,2020,8,29,0
2,2018-02-27,5,0,0,659759,0,0,107,417933,Five Stars has dramatically improved my toe na...,2018,2,27,1
3,2012-05-16,5,0,1,73493,0,0,66,166465,Great portable irrigator I recently got braces...,2012,5,16,1
4,2015-01-05,4,0,0,714841,0,0,14,289006,Four Stars The Salad Shooter works great and s...,2015,1,5,1


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import string, nltk
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
nltk.download('wordnet')
nltk.download('omw-1.4')

import nltk
nltk.download('stopwords')
import nltk
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 391875 entries, 0 to 392425
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   review_date       391875 non-null  datetime64[ns]
 1   review_rating     391875 non-null  int64         
 2   number_of_photos  391875 non-null  int64         
 3   helpful_vote      391875 non-null  int64         
 4   reviewer_ID       391875 non-null  object        
 5   fake_asin         391875 non-null  int64         
 6   fake_review       391875 non-null  int64         
 7   product_ID        391875 non-null  int64         
 8   review_ID         391875 non-null  int64         
 9   review            391875 non-null  object        
 10  year              391875 non-null  int64         
 11  month             391875 non-null  int64         
 12  day               391875 non-null  int64         
 13  is_weekday        391875 non-null  int64         
dtypes: d

In [ ]:
df.isnull().sum()

review_date         0
review_rating       0
number_of_photos    0
helpful_vote        0
reviewer_ID         0
fake_asin           0
fake_review         0
product_ID          0
review_ID           0
review              0
year                0
month               0
day                 0
is_weekday          0
dtype: int64

In [ ]:
# delete all gibberish in the review column
import re
df['review'] = df['review'].replace({r'[^a-zA-Z0-9\s\']':''}, regex=True)
#delete punctuation in review column
df['review'] = df['review'].apply(lambda x: re.sub(r'[^\w\s\']', '', x) if isinstance(x, str) else '')

In [ ]:
# format text data into lower structure.
df['review'] = df['review'].str.lower()

In [ ]:
# Define a function to remove stopwords
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = nltk.word_tokenize(text)
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

# Apply the function to the text column of the dataframe
df['review'] = df['review'].apply(remove_stopwords)

In [ ]:
stemmer = SnowballStemmer(language='english')

In [ ]:
# apply snowball stemming to eliminate affixes
def stem_words(text):
    words = nltk.word_tokenize(text)
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)
# Apply the function to the text column of the dataframe
df['review'] = df['review'].apply(stem_words)

In [ ]:
df.head()

,review_date,review_rating,number_of_photos,helpful_vote,reviewer_ID,fake_asin,fake_review,product_ID,review_ID,review,year,month,day,is_weekday
0,2020-04-13,5,0,0,380263,1,1,389,34510,super product husband hard time sleep first ni...,2020,4,13,1
1,2020-08-29,5,0,0,845137,0,0,99,381688,sturdi rving,2020,8,29,0
2,2018-02-27,5,0,0,659759,0,0,107,417933,five star dramat improv toe nail appear,2018,2,27,1
3,2012-05-16,5,0,1,73493,0,0,66,166465,great portabl irrig recent got brace need work...,2012,5,16,1
4,2015-01-05,4,0,0,714841,0,0,14,289006,four star salad shooter work great save time,2015,1,5,1


In [ ]:
from textblob import TextBlob
# create a function to compute the sentiment score of a text input
def get_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity, blob.sentiment.subjectivity

df[['polarity', 'subjectivity']] = df['review'].apply(get_sentiment).apply(pd.Series)

df.head()

,review_date,review_rating,number_of_photos,helpful_vote,reviewer_ID,fake_asin,fake_review,product_ID,review_ID,review,year,month,day,is_weekday,polarity,subjectivity
0,2020-04-13,5,0,0,380263,1,1,389,34510,super product husband hard time sleep first ni...,2020,4,13,1,0.315278,0.565278
1,2020-08-29,5,0,0,845137,0,0,99,381688,sturdi rving,2020,8,29,0,0.000000,0.000000
2,2018-02-27,5,0,0,659759,0,0,107,417933,five star dramat improv toe nail appear,2018,2,27,1,0.000000,0.000000
3,2012-05-16,5,0,1,73493,0,0,66,166465,great portabl irrig recent got brace need work...,2012,5,16,1,0.269388,0.631293
4,2015-01-05,4,0,0,714841,0,0,14,289006,four star salad shooter work great save time,2015,1,5,1,0.800000,0.750000


In [ ]:
# create a new variable "labeled_product_id" to label the product with fake review assumption. 
df['labeled_product_id'] = df['fake_asin'] * df ['product_ID']
df.head()

,review_date,review_rating,number_of_photos,helpful_vote,reviewer_ID,fake_asin,fake_review,product_ID,review_ID,review,year,month,day,is_weekday,polarity,subjectivity,labeled_product_id
0,2020-04-13,5,0,0,380263,1,1,389,34510,super product husband hard time sleep first ni...,2020,4,13,1,0.315278,0.565278,389
1,2020-08-29,5,0,0,845137,0,0,99,381688,sturdi rving,2020,8,29,0,0.000000,0.000000,0
2,2018-02-27,5,0,0,659759,0,0,107,417933,five star dramat improv toe nail appear,2018,2,27,1,0.000000,0.000000,0
3,2012-05-16,5,0,1,73493,0,0,66,166465,great portabl irrig recent got brace need work...,2012,5,16,1,0.269388,0.631293,0
4,2015-01-05,4,0,0,714841,0,0,14,289006,four star salad shooter work great save time,2015,1,5,1,0.800000,0.750000,0


In [ ]:
df.to_csv('train_with_pid.csv', index=False)

In [ ]:
test = pd.read_csv('/content/gdrive/MyDrive/test_reviews.csv',encoding='ISO-8859-1')

In [ ]:
test.info()

In [ ]:
test['review'] = test['review_title'] + ' '+test['review_body']

In [ ]:
test['review'] = test['review'].fillna('NA')

In [ ]:
# convert 'date' column to datetime object
test['review_date'] = pd.to_datetime(test['review_date'])

# create new 'year' column extracted from 'date' column
test['year'] = test['review_date'].dt.year

test['month'] = test['review_date'].dt.month

test['day'] = test['review_date'].dt.day

In [ ]:
test['is_weekday'] = pd.to_datetime(test['review_date']).dt.dayofweek.apply(lambda x: 1 if x < 5 else 0)

In [ ]:
test = test.drop(["review_title", "review_body","review_date"], axis = 1)

In [ ]:
test

,review_rating,number_of_photos,helpful_vote,reviewer_ID,fake_asin,product_ID,review_ID,review,year,month,day,is_weekday
0,4,1,0,440934,0,281,427196,Fit of phone 7+ I have the iPhone 7+ and it sa...,2019,9,4,1
1,5,0,0,1044859,0,17,205467,Sturdy and simply This product comes assembled...,2020,8,22,0
2,5,0,0,377805,1,342,202189,Great product I really like this product becau...,2020,5,29,1
3,1,0,2,747580,0,212,56726,Item got good reviews but ... I couldn't get t...,2019,9,7,0
4,5,0,0,1170571,0,10,409558,Great Sheet Protectors - Love the Easy Loading...,2012,10,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...
43635,5,0,0,1211822,0,355,76495,Works out perfectly! Perfect for what I needed...,2021,1,2,0
43636,5,0,0,266849,0,443,402464,Nice product and exactly what I was looking fo...,2020,6,8,1
43637,5,0,0,1046881,0,48,431941,Easy to read and accurate. Easy to read.,2020,1,6,1
43638,5,0,0,93064,0,114,262452,"This one is great, the rotation is smooth and ...",2016,8,30,1


In [ ]:
# delete all gibberish in the review column
test['review'] = test['review'].replace({r'[^a-zA-Z0-9\s\']':''}, regex=True)
#delete punctuation in review column
test['review'] = test['review'].apply(lambda x: re.sub(r'[^\w\s\']', '', x) if isinstance(x, str) else '')
test['review'] = test['review'].str.lower()

In [ ]:
# Define a function to remove stopwords
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = nltk.word_tokenize(text)
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

# Apply the function to the text column of the dataframe
test['review'] = test['review'].apply(remove_stopwords)

In [ ]:
# apply snowball stemming to eliminate affixes
def stem_words(text):
    words = nltk.word_tokenize(text)
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)
# Apply the function to the text column of the dataframe
test['review'] = test['review'].apply(stem_words)

In [ ]:
from textblob import TextBlob
# create a function to compute the sentiment score of a text input
def get_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity, blob.sentiment.subjectivity

test[['polarity', 'subjectivity']] = test['review'].apply(get_sentiment).apply(pd.Series)


In [ ]:
test['labeled_product_id'] = test['fake_asin'] * test ['product_ID']

In [ ]:
test.to_csv('test.csv',index=False)